## P&G Enterprise Migration RFP Analysis
Analysis notebook for P&G Enterprise RFP

In [1]:
# Load libraries
import pandas as pd
import json
import numpy as py

# Initialize variables
fileInput='cmdb.csv'

# Open input file, read into frame
dfCMDB = pd.read_csv(fileInput, keep_default_na=False)
dfCMDB.head()

,Scope,Node Name,Affinity,RFP App Name,Instance,Entity Type,Phy Loc,Target Region,Consuming Customer,Current Region,...,Operational Backups Policy,Resilliency / DR Classification,Current State Services,DB Instance Count,Shared Env App Count,Other Software,Validated Server,Encryption,SOX Critical,External Access
0,Primary,GADC-AADC02,AADC,-,,Virtual,ALPHARETTA - ALPHARETTA NGDC - BT567,NA,0,,...,In Region Backup,0,HP Internal Use,1,0,,N,Y,N,
1,Primary,GADC-AADCDEV03,AADC,0,,Virtual,ALPHARETTA - ALPHARETTA NGDC - BT567,EU,N/A,EMEA,...,Secondary Region Backup,0,Prod,,0,,N,Y,0,
2,Primary,WYN-AADC01,AADC,0,,Virtual,HP WYNYARD DATA CENTRE,EU,N/A,EMEA,...,Secondary Region Backup,0,Prod,,0,,N,Y,0,
3,Primary,WYN-AADC02,AADC,0,,Virtual,HP WYNYARD DATA CENTRE,EU,N/A,EMEA,...,Secondary Region Backup,0,Prod,1,0,,N,Y,N,
4,Primary,BDC-AADSYNC01,AADSYN,0,,Virtual,BLUE ASH DATA CENTER-BDC,NA,CF-IT,NA,...,In Region Backup,0,Prod,,0,,N,Y,0,


In [2]:
# List the fields
dfCMDB.dtypes

Scope                               object
Node Name                           object
Affinity                            object
RFP App Name                        object
Instance                            object
Entity Type                         object
Phy Loc                             object
Target Region                       object
Consuming Customer                  object
Current Region                      object
CPU                                float64
Avg CPU Load                       float64
Peak CPU Load                      float64
CPU Speed (MHz)                     object
Processing Type                     object
Mem (MB)                           float64
Avg % Mem Used                     float64
Peak Mem Used                      float64
Storage Type                        object
Alloc Size (GB)                      int64
Used Size (GB)                       int64
Platform                            object
OS Ver                              object
DB Rel/Ver 

In [3]:
# Count servers by Target Region
dfByTargetRegion=dfCMDB.groupby('Target Region', as_index=True).agg({'Target Region': pd.Series.count})
dfByTargetRegion.sort_values('Target Region', ascending=False)

,Target Region
Target Region,
NA,2140
EU,131
AP,73


In [4]:
#Add RDS Column
dfCMDB['RDS']=False
dfCMDB['RDS'].head()

0    False
1    False
2    False
3    False
4    False
Name: RDS, dtype: bool

In [5]:
# List the servers with database instances along with type
#pdCMDB['DB Instance Count']
dfDBServers = dfCMDB[dfCMDB['DB Instance Count'] != ""]
dfDBServers = dfDBServers[dfDBServers['DB Instance Count'] !="0"]
dfDBServers[['Node Name', 'DB Rel/Ver','Target Region', 'Resilliency / DR Classification']]

indexes=dfDBServers.index.tolist()

for row in indexes:
    dfCMDB.loc[row, 'RDS'] = True
    
dfCMDB.head()

,Scope,Node Name,Affinity,RFP App Name,Instance,Entity Type,Phy Loc,Target Region,Consuming Customer,Current Region,...,Resilliency / DR Classification,Current State Services,DB Instance Count,Shared Env App Count,Other Software,Validated Server,Encryption,SOX Critical,External Access,RDS
0,Primary,GADC-AADC02,AADC,-,,Virtual,ALPHARETTA - ALPHARETTA NGDC - BT567,NA,0,,...,0,HP Internal Use,1,0,,N,Y,N,,True
1,Primary,GADC-AADCDEV03,AADC,0,,Virtual,ALPHARETTA - ALPHARETTA NGDC - BT567,EU,N/A,EMEA,...,0,Prod,,0,,N,Y,0,,False
2,Primary,WYN-AADC01,AADC,0,,Virtual,HP WYNYARD DATA CENTRE,EU,N/A,EMEA,...,0,Prod,,0,,N,Y,0,,False
3,Primary,WYN-AADC02,AADC,0,,Virtual,HP WYNYARD DATA CENTRE,EU,N/A,EMEA,...,0,Prod,1,0,,N,Y,N,,True
4,Primary,BDC-AADSYNC01,AADSYN,0,,Virtual,BLUE ASH DATA CENTER-BDC,NA,CF-IT,NA,...,0,Prod,,0,,N,Y,0,,False


In [6]:
# Sum the DR classifications
dfDBDR=dfDBServers.groupby('Resilliency / DR Classification', as_index=True).agg({'Resilliency / DR Classification': pd.Series.count})
dfDBDR.sort_values('Resilliency / DR Classification', ascending=False)

,Resilliency / DR Classification
Resilliency / DR Classification,
0,240
N,45
AIS ADR,5
Y,3


# Issue
In the above, what do the DR classfications mean?

In [7]:
# Calculate cores needed from peak CPU
dfCMDB['cores_calc'] = (dfCMDB['CPU'] * dfCMDB['Peak CPU Load']) + .51
dfCMDB['cores_calc']=dfCMDB['cores_calc'].round(decimals=0)
dfCMDB[['CPU', 'Peak CPU Load', 'cores_calc']].head()

,CPU,Peak CPU Load,cores_calc
0,4.0,0.37,2.0
1,2.0,0.00,1.0
2,2.0,1.00,3.0
3,8.0,0.99,8.0
4,4.0,0.97,4.0


In [8]:
# Check the resulting peak memory
dfCMDB['Peak Mem Used'].head()

0    15.28
1     0.00
2     6.34
3    15.37
4     5.92
Name: Peak Mem Used, dtype: float64

In [9]:
# Correct missing peak memory
for index in dfCMDB.index.tolist():
    if dfCMDB.loc[index, 'Peak Mem Used'] == 0:
        dfCMDB.loc[index, 'Peak Mem Used'] = dfCMDB.loc[index, 'Mem (MB)'] / 1000
        
dfCMDB['Peak Mem Used'].head()

0    15.280
1     4.096
2     6.340
3    15.370
4     5.920
Name: Peak Mem Used, dtype: float64

In [10]:
# Create family inference column
dfCMDB['calc_family'] = ""

In [11]:
# Show unique environments
dfByEnv=dfDBServers.groupby('Current State Services', as_index=True).agg({'Current State Services': pd.Series.count})
dfByEnv

,Current State Services
Current State Services,
,19
0,12
AIS,13
Dev,24
Dev/QA,1
HP Internal Use,3
OBRU,2
Prod,172
QA,14


In [12]:
# Create memory to CPU ratio column
dfCMDB['mem_cpu_ratio'] = dfCMDB['Peak Mem Used'] / dfCMDB['cores_calc']
dfCMDB['mem_cpu_ratio'].head()

0    7.640000
1    4.096000
2    2.113333
3    1.921250
4    1.480000
Name: mem_cpu_ratio, dtype: float64

In [13]:
# Make an inference for ec2 family

for index in dfCMDB.index.tolist():
    if ((dfCMDB.loc[index, 'cores_calc'] <= 8) and (dfCMDB.loc[index, 'Peak Mem Used'] <=32)
        and ("Dev" in dfCMDB.loc[index, 'Current State Services']
             or "QA" in dfCMDB.loc[index, 'Current State Services']
             or "Test" in dfCMDB.loc[index, 'Current State Services'])):
            dfCMDB.loc[index, 'calc_family'] = "t"
            
    elif (dfCMDB.loc[index, 'mem_cpu_ratio'] < 3.5):
             dfCMDB.loc[index, 'calc_family'] = "c"
            
    elif (dfCMDB.loc[index, 'mem_cpu_ratio'] > 4.5):
             dfCMDB.loc[index, 'calc_family'] = "r"
            
    else:
             dfCMDB.loc[index, 'calc_family'] = "m"
             
dfCMDB[['calc_family', 'mem_cpu_ratio', 'CPU', 'Peak Mem Used']].head()

,calc_family,mem_cpu_ratio,CPU,Peak Mem Used
0,r,7.640000,4.0,15.280
1,m,4.096000,2.0,4.096
2,c,2.113333,2.0,6.340
3,c,1.921250,8.0,15.370
4,c,1.480000,4.0,5.920


In [14]:
# Report servers by source region
dfByTargetRegion=dfCMDB.groupby('Target Region', as_index=True).agg({'Target Region': pd.Series.count})
dfByTargetRegion

,Target Region
Target Region,
AP,73
EU,131
NA,2140


In [15]:
# Create AWS Region Column and map to source region
# ap-southeast-1
# us-east-1
# eu-central-1
dfCMDB['AWS_Region'] = ""

for index in dfCMDB.index.tolist():
    if dfCMDB.loc[index, 'Target Region'] == "AP":
        dfCMDB.loc[index, 'AWS_Region'] = "ap-southeast-1"
    elif dfCMDB.loc[index, 'Target Region'] == "EU":
        dfCMDB.loc[index, 'AWS_Region'] = "eu-central-1"
    else:
        dfCMDB.loc[index, 'AWS_Region'] = "us-east-1"
        
dfCMDB[['Target Region', 'AWS_Region']].head()

,Target Region,AWS_Region
0,NA,us-east-1
1,EU,eu-central-1
2,EU,eu-central-1
3,EU,eu-central-1
4,NA,us-east-1


In [16]:
# Determine unique platforms
dfByPlatform=dfCMDB.groupby('Platform', as_index=True).agg({'Platform': pd.Series.count})
dfByPlatform

,Platform
Platform,
LINUX,406
UNIX,27
VIRTUAL,5
WINDOWS,1906


In [17]:
# Check the count of servers by EC2 instance family
dfByFamily=dfCMDB.groupby('calc_family', as_index=True).agg({'calc_family': pd.Series.count})
dfByFamily

,calc_family
calc_family,
c,783
m,182
r,1130
t,249


In [18]:
# Check the source list of OS
dfByOS=dfCMDB.groupby('OS Ver', as_index=True).agg({'OS Ver': pd.Series.count})
dfByOS

,OS Ver
OS Ver,
,16
release 6.5,2
0,88
2012STD,1
5.2.3790,78
6..6002,6
6.0.6002,575
6.1.7600,10
6.1.7601,226


## Issue
There is no pattern to OS!  Will need to make inferences

In [19]:
# Create AWS OS column, search for key words in source os to map to EC2 platform
dfCMDB['AWS_OS'] = ""

for index in dfCMDB.index.tolist():
    if "WINDOWS" in dfCMDB.loc[index, 'Platform']:
        dfCMDB.loc[index, 'AWS_OS'] = "Windows"
    elif "LINUX" in dfCMDB.loc[index, 'Platform']:
        if ("RHEL" in dfCMDB.loc[index, 'OS Ver'] or
            "Red" in dfCMDB.loc[index, 'OS Ver'] or
            "RED" in dfCMDB.loc[index, 'OS Ver']):
                dfCMDB.loc[index, 'AWS_OS'] = "RHEL"
        else:
            dfCMDB.loc[index, 'AWS_OS'] = "Linux"
    else:
        dfCMDB.loc[index, 'AWS_OS'] = "Linux"

In [20]:
# Look at a report of resulting EC2 OS
dfAWSOS=dfCMDB.groupby('AWS_OS', as_index=True).agg({'AWS_OS': pd.Series.count})
dfAWSOS

,AWS_OS
AWS_OS,
Linux,108
RHEL,330
Windows,1906


In [21]:
# Core matching and pricing code
# Match calculated capacity requirements to EC2 instance types
# Price resulting EC2 instance types by hour, year, and 3-year RIs

import boto3
import json
import re
import pandas

regions = {"us-east-1", "eu-central-1", "ap-southeast-1"}
families = {"m", "c", "r", "t"}
oses = {"Windows", "RHEL", "Linux"}

for region in regions:
    for os in oses:
        for family in families:
            # Lets get specific and only get the license included, no pre-installed software, current generation, etc.
            client = boto3.client('pricing')
            
            if region == "us-east-1":
                location = "US East (N. Virginia)"
            elif region == "eu-central-1":
                location = "EU (Frankfurt)"
            else:
                location = "Asia Pacific (Singapore)"
            
            if family == "c":
                instanceFamily = "Compute optimized"
            elif family == "r":
                instanceFamily = "Memory optimized"
            else:
                instanceFamily = "General purpose"
    
            response = client.get_products(
                Filters=[
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'location',
                        'Value': location
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'operatingSystem',
                        'Value': os
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'instanceFamily',
                        'Value': instanceFamily
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'currentGeneration',
                        'Value': 'Yes'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'licenseModel',
                        'Value': 'No License required'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'tenancy',
                        'Value': 'Shared'
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'preInstalledSw',
                        'Value': 'NA'
                    }
                ],
                ServiceCode='AmazonEC2',
                MaxResults=100
            )
        
            # Let's parse the JSON and get the elements we need to map to EC2 instance type
            items=response['PriceList']


            # Lets make a dataframe with the EC2 instance choices that are rhel and memory optimized
            d={'instanceType':[], 'memory':[], 'family':[], 'one_hr_rate':[], 'one_yr_rate':[], 'three_yr_rate':[]}
            dfInstanceList=pd.DataFrame(data=d)
            index=0

            for item in items:
                jItem=json.loads(item)
                itemAttributes=jItem['product']['attributes']
                instanceType=itemAttributes['instanceType']
                instancefamily=instanceType[0:2]
                vcpu=itemAttributes['vcpu']
                sku=jItem['product']['sku']
                ondemandterm="JRTCKXETXF"
                ondemandratecode="6YS6EN2CT7"
                oneyearterm="6QCMYABX3D"
                oneyearratecode="2TG2D8R56U"
                threeyearterm="NQ3QZPMQV9"
                threeyearratecode="2TG2D8R56U"
                onehr_rate=jItem['terms']['OnDemand'][sku+"."+ondemandterm]['priceDimensions'][sku+"."+ondemandterm+"."+ondemandratecode]['pricePerUnit']['USD']
                oneyr_rate=jItem['terms']['Reserved'][sku+"."+oneyearterm]['priceDimensions'][sku+"."+oneyearterm+"."+oneyearratecode]['pricePerUnit']['USD']
                threeyr_rate=jItem['terms']['Reserved'][sku+"."+threeyearterm]['priceDimensions'][sku+"."+threeyearterm+"."+threeyearratecode]['pricePerUnit']['USD']
                
                # Drop the old lines
                if (family == "t"):
                    if (instancefamily[0] != "m"):
                        memoryelement=itemAttributes['memory']
                        memory=re.sub('[^0-9]','', memoryelement)
                        dfInstanceList.loc[index, 'instanceType'] = itemAttributes['instanceType']
                        dfInstanceList.loc[index, 'memory'] =  memory
                        dfInstanceList.loc[index,'family'] = family
                        dfInstanceList.loc[index, 'vcpu'] = vcpu 
                        dfInstanceList.loc[index, 'one_hr_rate'] = onehr_rate
                        dfInstanceList.loc[index, 'one_yr_rate'] = oneyr_rate
                        dfInstanceList.loc[index, 'three_yr_rate'] = threeyr_rate
                        index=index+1


                elif (region == "us-east-1"):
                    if ((instancefamily != "m4") and (instancefamily != "m3") and (instancefamily != "c4") and (instancefamily !="c3") and (instancefamily != "r3") and (instancefamily != "t2")):
                        memoryelement=itemAttributes['memory']
                        memory=re.sub('[^0-9]','', memoryelement)
                        dfInstanceList.loc[index, 'instanceType'] = itemAttributes['instanceType']
                        dfInstanceList.loc[index, 'memory'] =  memory
                        dfInstanceList.loc[index,'family'] = family
                        dfInstanceList.loc[index,'vcpu'] = vcpu
                        dfInstanceList.loc[index, 'one_hr_rate'] = onehr_rate
                        dfInstanceList.loc[index, 'one_yr_rate'] = oneyr_rate
                        dfInstanceList.loc[index, 'three_yr_rate'] = threeyr_rate
                        index=index+1

                else:
                    if ((instancefamily != "m3") and (instancefamily != "c3") and (instancefamily != "r3") and (instancefamily != "t2")):
                        memoryelement=itemAttributes['memory']
                        memory=re.sub('[^0-9]','', memoryelement)
                        dfInstanceList.loc[index, 'instanceType'] = itemAttributes['instanceType']
                        dfInstanceList.loc[index, 'memory'] =  memory
                        dfInstanceList.loc[index,'family'] = family
                        dfInstanceList.loc[index, 'vcpu'] = vcpu
                        dfInstanceList.loc[index, 'one_hr_rate'] = onehr_rate
                        dfInstanceList.loc[index, 'one_yr_rate'] = oneyr_rate
                        dfInstanceList.loc[index, 'three_yr_rate'] = threeyr_rate
                        index=index+1
            
            #Convert memory to numeric
            dfInstanceList['memory']=dfInstanceList['memory'].apply(pd.to_numeric)
            dfInstanceList['vcpu']=dfInstanceList['vcpu'].apply(pd.to_numeric)
            dfInstanceList['one_hr_rate']=dfInstanceList['one_hr_rate'].apply(pd.to_numeric)
            dfInstanceList['one_yr_rate']=dfInstanceList['one_yr_rate'].apply(pd.to_numeric)
            dfInstanceList['three_yr_rate']=dfInstanceList['three_yr_rate'].apply(pd.to_numeric)

            dfInstanceList_sorted=dfInstanceList.sort_values(by=['vcpu'], ascending=True)
            dfInstanceList_sorted=dfInstanceList_sorted.reset_index(drop=True)
                    
            dfCMDB_filter = dfCMDB[(dfCMDB.calc_family == family) & (dfCMDB.AWS_OS == os) & (dfCMDB.RDS == False) & (dfCMDB.AWS_Region == region)]

            # Map instances to EC2 instance types  

            for index in dfCMDB_filter.index.tolist():
                found=False
                instance=0

                while ((not(found)) & (instance < len(dfInstanceList_sorted))):
                    if ((dfInstanceList_sorted.loc[instance, 'memory'] >= dfCMDB.loc[index, 'Peak Mem Used']) and (dfInstanceList_sorted.loc[instance, 'vcpu'] >= dfCMDB.loc[index, 'cores_calc'])):
                        found = True
                        dfCMDB.loc[index, 'ec2_instance_type'] = dfInstanceList_sorted.loc[instance, 'instanceType']
                        dfCMDB.loc[index, 'one_hr_rate'] = dfInstanceList_sorted.loc[instance, 'one_hr_rate']
                        dfCMDB.loc[index, 'one_yr_rate'] = dfInstanceList_sorted.loc[instance, 'one_yr_rate']
                        dfCMDB.loc[index, 'three_yr_rate'] = dfInstanceList_sorted.loc[instance, 'three_yr_rate']

                    instance = instance +1


In [25]:
# Map source DB to AWS_DB

dfCMDB['AWS_DB'] = "" 
dfCMDB_filterrds = dfCMDB[(dfCMDB.RDS == True)]   
   
indexes=dfCMDB_filterrds.index.tolist()
for row in indexes:
    if "Oracle" in dfCMDB.loc[row, 'DB Rel/Ver']:
        dfCMDB.loc[row, 'AWS_DB'] = "Oracle"
    elif "SQL " in dfCMDB.loc[row, 'DB Rel/Ver']:
        dfCMDB.loc[row, 'AWS_DB'] = "SQL Server"
    else:
        dfCMDB.loc[row, 'AWS_DB'] = "Aurora MySQL"

# Issue
There are servers marked with Oracle but with zero instances - assuming just a normal server

In [26]:
# Map RDS instances

import boto3
import json
import re
import pandas

regions = {"us-east-1", "eu-central-1", "ap-northeast-2"}
families = {"m", "c", "r", "t"}
dbs = {"Oracle", "SQL Server", "Aurora MySQL"}

for region in regions:
    for db in dbs:
        for family in families:
            # Lets get specific and only get the license included, no pre-installed software, current generation, etc.
            client = boto3.client('pricing')
            
            if region == "us-east-1":
                location = "US East (N. Virginia)"
            elif region == "eu-central-1":
                location = "EU (Frankfurt)"
            else:
                location = "Asia Pacific (Seoul)"
            
            if family == "c":
                instanceFamily = "General purpose"
            elif family == "r":
                instanceFamily = "Memory optimized"
            else:
                instanceFamily = "General purpose"
                
            if db == "Aurora MySQL":
                licensemodel="No license required"
                instanceFamily = "Memory optimized"
            else:
                licensemodel="License included"
    
            response = client.get_products(
                Filters=[
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'location',
                        'Value': location
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'databaseEngine',
                        'Value': db
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'currentGeneration',
                        'Value': 'Yes'            
                    },
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'instanceFamily',
                        'Value': instanceFamily          
                    },      
                    {
                        'Type': 'TERM_MATCH',
                        'Field': 'licenseModel',
                        'Value': licensemodel            
                    }, 
                    ],                
                    ServiceCode='AmazonRDS',
                    MaxResults=100
                )
        
            # Let's parse the JSON and get the elements we need to map to EC2 instance type
            items=response['PriceList']

            # Lets make a dataframe with the RDS instance choices
            d={'instanceType':[], 'memory':[], 'family':[], 'one_hr_rate':[], 'one_yr_rate':[], 'three_yr_rate':[]}
            dfInstanceList=pd.DataFrame(data=d)
            index=0

            for item in items:
                jItem=json.loads(item)
                itemAttributes=jItem['product']['attributes']
                instanceType=itemAttributes['instanceType']
                instancefamily=instanceType[3:5]
                vcpu=itemAttributes['vcpu']
                sku=jItem['product']['sku']
                ondemandterm="JRTCKXETXF"
                ondemandratecode="6YS6EN2CT7"
                
                if (db == "SQL Server"):
                    oneyearterm="HU7G6KETJZ"
                else:
                    oneyearterm="6QCMYABX3D"
                
                oneyearratecode="2TG2D8R56U"
                onyearratecode="6YS6EN2CT7"
                threeyearterm="NQ3QZPMQV9"
                threeyearratecode="2TG2D8R56U"
                onehr_rate=jItem['terms']['OnDemand'][sku+"."+ondemandterm]['priceDimensions'][sku+"."+ondemandterm+"."+ondemandratecode]['pricePerUnit']['USD']
                oneyr_rate=jItem['terms']['Reserved'][sku+"."+oneyearterm]['priceDimensions'][sku+"."+oneyearterm+"."+oneyearratecode]['pricePerUnit']['USD']
                
                # Account for the absence of an 'one-year all up-front' option for SQL server
                if db == "SQL Server":  
                    sqlhourly=float(jItem['terms']['Reserved'][sku+"."+oneyearterm]['priceDimensions'][sku+"."+oneyearterm+"."+ondemandratecode]['pricePerUnit']['USD'])
                    oneyr_rate = float(oneyr_rate) + (sqlhourly * 8760)

                threeyr_rate=jItem['terms']['Reserved'][sku+"."+threeyearterm]['priceDimensions'][sku+"."+threeyearterm+"."+threeyearratecode]['pricePerUnit']['USD']
           
                # person = input('Enter your name: ')
        
                # Load rates
                memoryelement=itemAttributes['memory']
                memory=re.sub('[^0-9]','', memoryelement)
                dfInstanceList.loc[index, 'instanceType'] = itemAttributes['instanceType']
                dfInstanceList.loc[index, 'memory'] =  memory
                dfInstanceList.loc[index,'family'] = family
                dfInstanceList.loc[index, 'vcpu'] = vcpu 
                dfInstanceList.loc[index, 'one_hr_rate'] = onehr_rate
                dfInstanceList.loc[index, 'one_yr_rate'] = oneyr_rate
                dfInstanceList.loc[index, 'three_yr_rate'] = threeyr_rate
                index=index+1
            
            #Convert memory to numeric
            
            dfInstanceList['memory']=dfInstanceList['memory'].apply(pd.to_numeric)
            dfInstanceList['vcpu']=dfInstanceList['vcpu'].apply(pd.to_numeric)
            dfInstanceList['one_hr_rate']=dfInstanceList['one_hr_rate'].apply(pd.to_numeric)
            dfInstanceList['one_yr_rate']=dfInstanceList['one_yr_rate'].apply(pd.to_numeric)
            dfInstanceList['three_yr_rate']=dfInstanceList['three_yr_rate'].apply(pd.to_numeric)

            dfInstanceList_sorted=dfInstanceList.sort_values(by=['vcpu'], ascending=True)
            dfInstanceList_sorted=dfInstanceList_sorted.reset_index(drop=True)
            
            if region == "ap-northeast-2":
                myregion = "ap-southeast-1"
            else: myregion = region
                    
            dfCMDB_filter = dfCMDB[(dfCMDB.calc_family == family) & (dfCMDB.AWS_DB == db) & (dfCMDB.RDS == True) & (dfCMDB.AWS_Region == myregion)]

            
            # Map instances to EC2 instance types  
            for index in dfCMDB_filter.index.tolist():
                found=False
                instance=0
                
                while ((not(found)) & (instance < len(dfInstanceList_sorted))):
                    if ((dfInstanceList_sorted.loc[instance, 'memory'] >= dfCMDB.loc[index, 'Peak Mem Used']) and (dfInstanceList_sorted.loc[instance, 'vcpu'] >= dfCMDB.loc[index, 'cores_calc'])):
                        found = True
                        dfCMDB.loc[index, 'ec2_instance_type'] = dfInstanceList_sorted.loc[instance, 'instanceType']
                        dfCMDB.loc[index, 'one_hr_rate'] = dfInstanceList_sorted.loc[instance, 'one_hr_rate']
                        dfCMDB.loc[index, 'one_yr_rate'] = dfInstanceList_sorted.loc[instance, 'one_yr_rate']
                        dfCMDB.loc[index, 'three_yr_rate'] = dfInstanceList_sorted.loc[instance, 'three_yr_rate']

                    instance = instance +1

In [27]:
# Compute EBS and snapshots
dfCMDB['ebs_month_rate'] = 0

# Flat storage rates assuming 1% monthly rate of change on EC2
ec2_ebs_unit_cost=.151
rds_ebs_unit_cost=.116

for index in dfCMDB_filter.index.tolist():
    if dfCMDB.loc[index, 'RDS'] == True:
        dfCMDB.loc[index, 'ebs_month_rate'] = dfCMDB.loc[index, 'Used Size (GB)'] * rds_ebs_unit_cost
        response=input('Stop')
    else:
        dfCMDB.loc[index, 'ebs_month_rate'] = dfCMDB.loc[index, 'Used Size (GB)'] * ec2_ebs_unit_cost

In [28]:
dfCMDB.dtypes

Scope                               object
Node Name                           object
Affinity                            object
RFP App Name                        object
Instance                            object
Entity Type                         object
Phy Loc                             object
Target Region                       object
Consuming Customer                  object
Current Region                      object
CPU                                float64
Avg CPU Load                       float64
Peak CPU Load                      float64
CPU Speed (MHz)                     object
Processing Type                     object
Mem (MB)                           float64
Avg % Mem Used                     float64
Peak Mem Used                      float64
Storage Type                        object
Alloc Size (GB)                      int64
Used Size (GB)                       int64
Platform                            object
OS Ver                              object
DB Rel/Ver 

In [29]:
dfCMDB.to_csv('bom.csv')